In [ ]:
from mpcontribs.client import load_client
from mpcontribs.io.core.recdict import RecursiveDict
from mpcontribs.io.core.components.hdata import HierarchicalData
from mpcontribs.io.core.components.tdata import Table
from mpcontribs.io.core.components.gdata import Plot

In [ ]:
api_key = 'your-api-key-here' # from https://portal.mpcontribs.org/dashboard
client = load_client(api_key)

## project info & overview table
see https://portal.mpcontribs.org/transparent_conductors

In [ ]:
project = 'transparent_conductors'
info = client.projects.get_entry(project=project).response().result
RecursiveDict(info)

In [ ]:
columns = ['doping', 'number_of_studies']
keys, subkeys = ['computed_gap'], ['hse06_band_gap', 'hse06_band_gap']
columns += ['.'.join([k, sk]) for k in keys for sk in subkeys]
data = client.projects.get_table(project=project, columns=columns).response().result
columns = list(data['items'][0].keys()) # reset to include units
Table(data['items'], columns=columns, project=project, api_key=api_key)

## specific contribution

In [ ]:
cid = '5d032b62e6afb7143d5270b6'
contrib = client.contributions.get_entry(cid=cid).response().result

### hierarchical data

In [ ]:
HierarchicalData(contrib['content']['data']) # dictionary

## query

In [ ]:
# see https://portal.mpcontribs.org/<project> and
# https://api.mpcontribs.org/#/contributions/get_entries

per_page = 20 # 200 is max retrievable contributions per page
fields = [ # which data fields to retrieve
    'doping', 'number_of_studies', 'computed_gap.hse06_band_gap', 'computed_gap.hse06_direct_gap'
]
mask = [f'content.data.{field}' for field in fields]
mask += ['content.data.formula', 'identifier']
filters = [
    'formula__contains:Cd',
    'computed_gap.hse06_band_gap__gt:2', 'computed_gap.hse06_band_gap__lt:3',
] # implicitly AND, only last filter kept for duplicate field/operator combinations

contribs = client.contributions.get_entries(
    projects=[project], filters=filters, mask=mask, per_page=per_page
).response().result # -> see pagination

print('found', len(contribs), 'materials')
contribs[0]

### pagination

In [ ]:
# only run this once you've optimized masks and filters to what you actually need (see above)
all_contribs = []
while 1:
    page = int(len(all_contribs) / per_page) + 1
    contribs = client.contributions.get_entries(
        projects=[project], filters=filters, mask=mask, per_page=per_page, page=page
    ).response().result
    all_contribs.extend(contribs)
    if not contribs or len(contribs) < per_page:
        break

print('found', len(all_contribs), 'materials')
all_contribs[-1]

### save data for download
Save this notebook after modifying and executing its code. Then click on the Jupyter logo at the top to get back to the file browser and select files for download. Note that Binder sessions are [ephemeral](https://mybinder.readthedocs.io/en/latest/faq.html#how-long-will-my-binder-session-last), so make sure to download this notebook (and your data) before closing the session.

In [ ]:
import json
with open('my_data.json', 'w') as f:
    json.dump(all_contribs, f)